In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [3]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [5]:
df["review"]=df["review"].str.lower()
df.head(2)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive


# remove html tags

In [14]:
import re

def removeTags(text):
    data = re.compile('<.*?>')
    return re.sub(data,"",text)

In [16]:
df["review"]=df["review"].apply(removeTags)

# remove urls


In [23]:
def removeUrls(text):
    data = re.compile(r'https?://\S+|www\.\S+') 
    return re.sub(data,"",text)

In [24]:
removeUrls("hellow join us at www.gogle.com")

'hellow join us at '

# Removing Punctuation

In [26]:
import string
exclude = string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
def removepunc(text):
    return text.translate(str.maketrans('','',exclude))

In [30]:
df["review"]=df["review"].apply(removepunc)

# Chatword treatment

In [35]:
chat_words_dict = {
    "hello": "A greeting or expression of goodwill",
    "brb": "Be right back",
    "lol": "Laugh out loud",
    "omg": "Oh my God",
    "idk": "I don't know",
    "btw": "By the way",
    "gtg": "Got to go",
    "ttyl": "Talk to you later",
    "np": "No problem",
    "thx": "Thanks",
    "yw": "You're welcome",
    "imo": "In my opinion",
    "fyi": "For your information",
    "asap": "As soon as possible",
    "bff": "Best friends forever",
    "tbh": "To be honest",
    "smh": "Shaking my head",
    "afaik": "As far as I know",
    "irl": "In real life",
    "dm": "Direct message",
    "rofl": "Rolling on the floor laughing",
    "lmao": "Laughing my ass off",
    "bbl": "Be back later",
    "gr8": "Great",
    "jk": "Just kidding",
    "nvm": "Never mind",
    "omw": "On my way",
    "ppl": "People",
    "pls": "Please",
    "thx": "Thanks",
    "u": "You",
    "ur": "Your",
    "w/e": "Whatever",
    "w/o": "Without",
    "w8": "Wait",
    "xoxo": "Hugs and kisses",
    "yolo": "You only live once",
    "b4": "Before",
    "cya": "See you",
    "ez": "Easy",
    "faq": "Frequently asked questions",
    "ftw": "For the win",
    "gg": "Good game",
    "gl": "Good luck",
    "hf": "Have fun",
    "hmu": "Hit me up",
    "ic": "I see",
    "imo": "In my opinion",
    "irl": "In real life",
    "jk": "Just kidding",
    "k": "Okay",
    "lmk": "Let me know",
    "msg": "Message",
    "n/a": "Not applicable",
    "np": "No problem",
    "omg": "Oh my God",
    "plz": "Please",
    "q": "Question",
    "r": "Are",
    "rn": "Right now",
    "sry": "Sorry",
    "tba": "To be announced",
    "tbd": "To be decided",
    "tgif": "Thank God it's Friday",
    "thx": "Thanks",
    "ttyl": "Talk to you later",
    "u": "You",
    "ur": "Your",
    "w/": "With",
    "w/o": "Without",
    "w8": "Wait",
    "xoxo": "Hugs and kisses",
    "yolo": "You only live once",
    "b4": "Before",
    "cya": "See you",
    "ez": "Easy",
    "faq": "Frequently asked questions",
    "ftw": "For the win",
    "gg": "Good game",
    "gl": "Good luck",
    "hf": "Have fun",
    "hmu": "Hit me up",
    "ic": "I see",
    "imo": "In my opinion",
    "irl": "In real life",
    "jk": "Just kidding",
    "k": "Okay",
    "lmk": "Let me know",
    "msg": "Message",
    "n/a": "Not applicable",
    "np": "No problem",
    "omg": "Oh my God",
    "plz": "Please",
    "q": "Question",
    "r": "Are",
    "rn": "Right now",
    "sry": "Sorry",
    "tba": "To be announced",
    "tbd": "To be decided",
    "tgif": "Thank God it's Friday",
    "thx": "Thanks",
    "ttyl": "Talk to you later",
    "u": "You",
    "ur": "Your",
    "w/": "With",
    "w/o": "Without",
    "w8": "Wait",
    "xoxo": "Hugs and kisses",
    "yolo": "You only live once"
}
chat_words_dict

AttributeError: 'dict' object has no attribute 'toarray'

In [43]:
def ChatWwordsTreatment(text):
    new_text=[]
    for w in text.split():
        if w in chat_words_dict:
            new_text.append(chat_words_dict[w])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [44]:
ChatWwordsTreatment("tbd i am going to gemrany")

'To be decided i am going to gemrany'

In [46]:
ChatWwordsTreatment("nvm i am going to gemrany")

'Never mind i am going to gemrany'

# Spelling Correction

In [52]:
from textblob import TextBlob

incorrect_sentence="certain candetion are vory bade"

correctcion = TextBlob(incorrect_sentence)

correction.correct().string

'certain condition are very bade'

# Stop_Words_Removing

In [74]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [83]:
def removeStopwords(text):
    new_text=[]
    for w in text.split():
        if w in stopwords.words("english"):
            new_text.append("")
        else:
            new_text.append(w)
    return " ".join(new_text)

In [86]:
df["review"]=df["review"].apply(removeStopwords)
df["review"].head(20)

KeyboardInterrupt: 

# Handling_Emjoies

In [87]:
import re

def remove_emojies(text):
    emojies_pattren = re.compile("["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
                                )
    text = re.sub(emojies_pattren,"",text)
    return text

In [90]:
remove_emojies("Hello 😊! How are you doing today? 🚀")

'Hello ! How are you doing today? '

In [91]:
import emoji
print(emoji.demojize("Hello 😊! How are you doing today? 🚀"))

Hello :smiling_face_with_smiling_eyes:! How are you doing today? :rocket:


# Tokenization 
# 1: Word Token
# 2: Sentence Token

In [92]:
sentnce = "i am going to islamabd"
sentnce.split()

['i', 'am', 'going', 'to', 'islamabd']

In [99]:
sentnce = "i am going to islamabd.my friend is moving to lahore"
sentnce.split(".")

['i am going to islamabd', 'my friend is moving to lahore']

In [113]:
#Nltk

from nltk.tokenize import word_tokenize,sent_tokenize
sent = "i have ph.D  in A.i"
word_tokenize(sent)

#Spacy is also a best option for tokenization

import spacy
nlp = spacy.load("en_core_web_sm")

sent2 = "mail us at warishayat69@gmail.com"

doc1 = nlp(sent2)

for token in doc1:
    print(token)

mail
us
at
warishayat69@gmail.com


# Lemmatizer/Stemming

In [125]:
import nltk
from nltk.stem import WordNetLemmatizer
lemitizer = WordNetLemmatizer()
sentence = "we as running and eating at the same time. we had habit of stemming after playing long hour sin the son"
punctuation = ".?!,:;"
sent_word = nltk.word_tokenize(sentence)
for word in sent_word:
    if word in punctuation:
        sent_word.remove(word)
        
print("{0:20}{0:20}".format("word","Lemma"))
for word in sent_word:
    print("{0:20}{0:20}".format(word,lemitizer.lemmatize(word,pos="v")))

word                word                


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************